1. Figure out which examples googlenet gets right, and which ones it gets wrong from the test set
2. Use TCAV to generate explanations for the wrong ones
3. Analyze the explanations

In [3]:
# ..........torch imports............
from pathlib import Path

import numpy as np
import torch
import torchvision
import os
import glob

from PIL import Image
from torchvision.datasets import ImageFolder
# .... Captum imports..................
from captum.attr import LayerIntegratedGradients
from captum.concept import TCAV
from captum.concept._utils.common import concepts_to_str

# .... Local imports..................
from joblib import load, dump

from HierarchicalExplanation import HierarchicalExplanation
from generate_data.hierarchy import Hierarchy
from utils import assemble_all_concepts_from_hierarchy, assemble_random_concepts, generate_experiments
from utils import load_image_tensors, transform, plot_tcav_scores, assemble_scores, get_pval, show_boxplots

# Load Hierarchy
HIERARCHY_JSON_PATH = 'generate_data/hierarchy.json'
HIERARCHY_WORDNET_PATH = 'generate_data/wordnet_labels.txt'
IMAGENET_IDX_TO_LABELS = 'generate_data/imagenet1000_clsidx_to_labels.txt'
h = Hierarchy(json_path=HIERARCHY_JSON_PATH, wordnet_labels_path=HIERARCHY_WORDNET_PATH,
              imagenet_idx_labels_path=IMAGENET_IDX_TO_LABELS)

###################################################
# Assemble Concepts
# Let's assemble concepts into Concept instances using Concept class and concept images stored in `concepts_path`.
###################################################

# concepts_path = "/home/devvrit/ishann/data/captum/tcav/concepts"
concepts_path = "../data"

# Assemble non-random concepts
concepts = assemble_all_concepts_from_hierarchy(h=h, num_images=100, concepts_path=concepts_path,
                                                recreate_if_exists=True)  # Only 100 images for testing, can increase later

# Assemble all random concepts
random_concepts = assemble_random_concepts(concepts_path=concepts_path)


# Defining GoogleNet Model
model = torchvision.models.googlenet(pretrained=True)
model = model.eval()
layers=['fc']

Assembling concepts...


Creating Random concepts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 394.35it/s]


In [15]:
# 1. Figure out which examples googlenet gets right, and which ones it gets wrong from the test set
import json
import tqdm

data_path = '../data'
IMAGENET_IDX_TO_LABELS = 'generate_data/imagenet1000_clsidx_to_labels.txt'
with open(IMAGENET_IDX_TO_LABELS, 'r') as f:
    idx2label = json.load(fp=f)

class_name = 'Siberian husky'
class_idx = h.imagenet_label2idx[class_name]
wrong_count = 1000

path = os.path.join(data_path, class_name)
filenames = glob.glob(path + '/*.JPEG')
correct_files = []
wrong_files = []
for i, filename in tqdm.notebook.tqdm(enumerate(filenames)):
    if len(wrong_files) >= wrong_count:
        break

    img = transform(Image.open(filename).convert('RGB'))
    img_batch = torch.unsqueeze(img, 0)
    pred = model(img_batch)
    if torch.argmax(pred) != class_idx:
        wrong_files.append(filename)
    else:
        correct_files.append(filename)

0it [00:00, ?it/s]

In [18]:
# Load the files into tensors
def load_image_tensors_from_list(list_images, do_transform=True, count=100):

    tensors = []
    for i, filename in enumerate(list_images):
        if i >= count:
            break

        img = Image.open(filename).convert('RGB')
        tensors.append(transform(img) if do_transform else img)
        
    return torch.stack(tensors)

class_images_correct = load_image_tensors_from_list(correct_files, count=100)
class_images_wrong = load_image_tensors_from_list(wrong_files, count=100)

In [22]:
# Explain the right and wrong predictions
he = HierarchicalExplanation(h=h, model=model, layer='fc', n_steps=5, load_save=False)

# Right
explanations = he.explain(input_tensors=class_images_correct, input_class_name=class_name, input_idx=class_idx, get_concepts_from_name=lambda x: concepts[x] if x in concepts else random_concepts[int(x.replace("random_", ""))])
print(explanations)
long_form = he.long_form_explanations(explanations, "Siberian husky")
print(long_form)

[{'level_name': 'entity', 'children': [('equipment', tensor(0.4858)), ('geological formation', tensor(-0.0154)), ('organism', tensor(0.1385)), ('random_0', tensor(-0.5575))], 'pval': 0.6078544147126332}, {'level_name': 'organism', 'children': [('canine', tensor(0.0014)), ('feline', tensor(0.1746)), ('random_1', tensor(-0.0894))], 'pval': 0.784246780077577}, {'level_name': 'canine', 'children': [('white wolf', tensor(-0.0280)), ('dog', tensor(0.0639)), ('random_2', tensor(-0.0229))], 'pval': 0.0008236621685874774}, {'level_name': 'dog', 'children': [('golden retriever', tensor(0.0135)), ('dalmatian', tensor(0.1094)), ('Siberian husky', tensor(0.1251)), ('random_3', tensor(-0.1142))], 'pval': 0.13241210533018588}]
The input is predicted to be a(n) Siberian husky (p-value: 0.1324).
It is predicted to be a(n) Siberian husky because it is a dog.
It is a(n) Siberian husky because out of all dogs, Siberian husky has the highest score among sub-classes:  (p-value: 0.1324)
Class Name 		 Score
g

In [21]:
# Wrong
explanations = he.explain(input_tensors=class_images_wrong, input_class_name=class_name, input_idx=class_idx, get_concepts_from_name=lambda x: concepts[x] if x in concepts else random_concepts[int(x.replace("random_", ""))])
print(explanations)
long_form = he.long_form_explanations(explanations, "Siberian husky")
print(long_form)

[{'level_name': 'entity', 'children': [('equipment', tensor(-0.0639)), ('geological formation', tensor(-0.0089)), ('organism', tensor(0.3964)), ('random_0', tensor(-0.3014))], 'pval': 0.004377852706947431}, {'level_name': 'organism', 'children': [('canine', tensor(0.8123)), ('feline', tensor(0.0281)), ('random_1', tensor(-0.4640))], 'pval': 0.052579513790523294}, {'level_name': 'canine', 'children': [('white wolf', tensor(0.1330)), ('dog', tensor(0.4865)), ('random_2', tensor(-0.3182))], 'pval': 0.1241489849560697}, {'level_name': 'dog', 'children': [('golden retriever', tensor(-0.0616)), ('dalmatian', tensor(-0.1182)), ('Siberian husky', tensor(0.5731)), ('random_3', tensor(-0.2777))], 'pval': 0.00036013454419289275}]
The input is predicted to be a(n) Siberian husky (p-value: 0.0004).
It is predicted to be a(n) Siberian husky because it is a dog.
It is a(n) Siberian husky because out of all dogs, Siberian husky has the highest score among sub-classes:  (p-value: 0.0004)
Class Name 		 

In [24]:
# Explain images from another class entirely
husky_idx = h.imagenet_label2idx[class_name]


class_images = load_image_tensors(class_name, root_path=concepts_path, transform=False, count=100)
class_tensors = torch.stack([transform(img) for img in class_images])
class_idx = h.imagenet_label2idx[class_name]


explanations = he.explain(input_tensors=class_tensors, input_class_name='Siberian husky', input_idx=husky_idx, get_concepts_from_name=lambda x: concepts[x] if x in concepts else random_concepts[int(x.replace("random_", ""))])
print(explanations)
long_form = he.long_form_explanations(explanations, "Siberian husky")
print(long_form)

[{'level_name': 'entity', 'children': [('equipment', tensor(0.0693)), ('geological formation', tensor(0.4898)), ('organism', tensor(0.1743)), ('random_0', tensor(-0.4526))], 'pval': 0.6374294499898574}, {'level_name': 'organism', 'children': [('canine', tensor(0.0435)), ('feline', tensor(-0.0337)), ('random_1', tensor(-0.0126))], 'pval': 0.02403836271634667}, {'level_name': 'canine', 'children': [('white wolf', tensor(-0.0413)), ('dog', tensor(0.1524)), ('random_2', tensor(-0.0399))], 'pval': 1.3732571329707236e-05}, {'level_name': 'dog', 'children': [('golden retriever', tensor(0.0345)), ('dalmatian', tensor(0.1032)), ('Siberian husky', tensor(0.0183)), ('random_3', tensor(-0.0708))], 'pval': 0.9408930790868546}]
The input is predicted to be a(n) Siberian husky (p-value: 0.9409).
It is predicted to be a(n) Siberian husky because it is a dog.
It is a(n) Siberian husky because out of all dogs, Siberian husky has the highest score among sub-classes:  (p-value: 0.9409)
Class Name 		 Score

The output you provided is from a trained GoogLeNet model that has been given images of a valley (the landform) and asked to explain how it is a Siberian husky. The model uses a hierarchical classification system, in which it first assigns scores to different classes at the highest level of the hierarchy, then repeats the process for sub-classes within those classes, and so on until it reaches the final prediction.

In this case, the model predicts that the input images are of a Siberian husky with a low degree of confidence (p-value: 0.9409). The model arrives at this prediction by first classifying the input as an organism, then a canine, then a dog, and finally as a Siberian husky. For each classification, the model assigns scores to different sub-classes within that category, and the sub-class with the highest score is chosen as the final prediction.

For example, within the category of canines, the model assigns a score of 0.0435 to the sub-class of canine and a score of -0.0337 to the sub-class of feline. Since the score for canine is higher, it is chosen as the final prediction for that category. This process is repeated for each classification until the final prediction of Siberian husky is reached.

However, the high p-values for each classification indicate that the model is not very confident in its predictions. A p-value of 0.9409 for the final prediction of Siberian husky is quite high, which means that there is a high probability that the observed result (the prediction of a Siberian husky) occurred by chance. This indicates that the model is not very confident in its prediction, and the result should be interpreted with caution.

Overall, this output shows how the trained GoogLeNet model uses a hierarchical classification system to arrive at a prediction, but the high p-values suggest that the model is not very confident in its final prediction.